In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io
import math

# format: virtualflies x (PPL1_01, 02, .., 05, odor)
# arranged odor #0 - #4
virtualFly = scipy.io.loadmat('virtualFly_MBON.mat')['virtualFly_MBON']

In [2]:
ODOR_NUM = 5
MBON_NUM = 6
NUM_FLIES_PER_ODOR = 12

virtualFly.shape

VF_OD1 = virtualFly[:NUM_FLIES_PER_ODOR**MBON_NUM,:]
VF_OD2 = virtualFly[NUM_FLIES_PER_ODOR**MBON_NUM:2*NUM_FLIES_PER_ODOR**MBON_NUM,:]
VF_OD3 = virtualFly[2*NUM_FLIES_PER_ODOR**MBON_NUM:3*NUM_FLIES_PER_ODOR**MBON_NUM,:]
VF_OD4 = virtualFly[3*NUM_FLIES_PER_ODOR**MBON_NUM:4*NUM_FLIES_PER_ODOR**MBON_NUM,:]
VF_OD5 = virtualFly[4*NUM_FLIES_PER_ODOR**MBON_NUM:5*NUM_FLIES_PER_ODOR**MBON_NUM,:]

'''
print(VF_OD1.shape)
print(VF_OD2.shape)
print(VF_OD3.shape)
print(VF_OD4.shape)
print(VF_OD5.shape)
'''

#shuffle data within the same odor
np.random.shuffle(VF_OD1)
np.random.shuffle(VF_OD2)
np.random.shuffle(VF_OD3)
np.random.shuffle(VF_OD4)
np.random.shuffle(VF_OD5)

In [14]:
# dataset arrange

numTrain = virtualFly.shape[0] * 0.9
numVal = virtualFly.shape[0] * 0.05
numTest = virtualFly.shape[0] * 0.05

'''
print(numTrain - numTrainPerOD*ODOR_NUM)
print(numVal - numValPerOD*ODOR_NUM)
print(numTest - numTestPerOD*ODOR_NUM)
print(virtualFly.shape[0]/5 - numTrainPerOD - numValPerOD - numTestPerOD)
print(numTrainPerOD, numValPerOD, numTestPerOD)
'''

'''
match numbers to those of PPL1
PPL1 dataset numbers
Train shape:  (1119740, 5)
                   
Validation shape:  (62205, 5)
Test shape:  (62205, 5)

train: 223948
val: 12441
test: 12441
'''

numTrainPerOD = 223948
numValPerOD = 12441
numTestPerOD = 12441



mask = range(numTrainPerOD)
trainSet = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 
np.random.shuffle(trainSet)
X_train = trainSet[:,:MBON_NUM]
y_train = trainSet[:,-1]-1 # -1 to conform python index from 0 to 4 (class imported from MATLAB was from 1 to 5)
X_train_shuffle = np.copy(X_train)
np.random.shuffle(X_train_shuffle)

mask = range(numTrainPerOD, numTrainPerOD+numValPerOD)
valSet = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 
np.random.shuffle(valSet)
X_val = valSet[:,:MBON_NUM]
y_val = valSet[:,-1]-1 # to conform python index format

mask = range(numTrainPerOD+numValPerOD, numTrainPerOD+numValPerOD+numTestPerOD)
testSet = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 
np.random.shuffle(testSet)
X_test = testSet[:,:MBON_NUM]
y_test = testSet[:,-1]-1 # to conform python index format



print(trainSet.shape)
print(valSet.shape)
print(testSet.shape)

print('Train shape: ', X_train.shape)
print('Validation shape: ', X_val.shape)
print('Test shape: ', X_test.shape)


(1119740, 7)
(62205, 7)
(62205, 7)
Train shape:  (1119740, 6)
Validation shape:  (62205, 6)
Test shape:  (62205, 6)


In [15]:
# preprocessing

#-mean - should do std as well?
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
print(mean,'\n', std)

X_train -= mean
X_train_shuffle -= mean
X_val -= mean
X_test -= mean

'''
X_train /= std
X_val /= std
X_test /= std
'''

[18.52240635  5.17894083  5.77221667  7.7397099  11.75307794 11.04826219] 
 [13.53912313  9.8297878   7.37823053  8.45913914  7.562449   12.24975283]


'\nX_train /= std\nX_val /= std\nX_test /= std\n'

In [23]:
def train(X, Y, XV, YV, model, lr, reg, batchSize, epoch):
    X = torch.Tensor(X)
    Y = torch.Tensor(Y)
    #X = torch.FloatTensor(X)
    #Y = torch.FloatTensor(Y)
    N = len(Y)
    XV = torch.FloatTensor(XV)
    YV = torch.FloatTensor(YV)

    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=lr)


   
    for epoch in range(epoch):
        perm = torch.randperm(N)
        sum_loss = 0
        model.train()
        for i in range(0, N, batchSize):
            x = X[perm[i:i+batchSize]]
            y = Y[perm[i:i+batchSize]]

            optimizer.zero_grad()
            weight = model.weight
            bias = model.bias
            output = model(x)
            
            #print(weight)
            #print(bias)
            #print(x)
            #print(x.double() @ weight + bias.double())
            #print(torch.mm(x, torch.t(model.weight))
            #print(torch.addmm(bias, x, torch.t(weight)))
            
            '''
            print("x:",x)
            print("y:",y.long())
            print("output:",output)
            '''
            
            correct_class_score = output[torch.arange(x.shape[0]), y.long()]
            
            margin = torch.maximum(torch.zeros(output.shape), output - correct_class_score.unsqueeze(1) + 1) # delta is 1
            loss = (torch.sum(margin)-1*x.shape[0])/x.shape[0] # remove correct case when delta is 1
            #print("loss1:", loss)
            loss += reg * torch.sum((weight.t() @ weight)) #/ 2.0
            #print("loss2:", loss)
            
            '''
            print(output - correct_class_score.unsqueeze(1)+1)
            print(torch.zeros(output.shape))
            print(correct_class_score)
            print("loss:", loss)
            '''
            #loss = torch.mean(torch.clamp(1 - y * output, min=0))
            
            loss.backward()
            optimizer.step()

            sum_loss += float(loss)
            

        print("Epoch: {:4d}\tloss: {}".format(epoch, sum_loss / N))

        numScore = 0
        model.eval()
        for i in range(0, len(YV), batchSize):
            x = XV[i:i+batchSize]
            y = YV[i:i+batchSize]
            
            optimizer.zero_grad()
            with torch.no_grad():
                output = model(x).numpy()
                pred = np.argmax(output, axis = 1)
                #print(pred)
                #print(y.numpy())
                numScore += np.sum((pred == y.numpy()))
                #print(numScore)
            
        
        print("validation acuuracy: {}".format(numScore / len(YV)))
        
def test(X, Y, model, batchSize, numOnly = False):
    XT = torch.FloatTensor(X)
    YT = torch.FloatTensor(Y)
    numScore = 0
    model.eval()
    for i in range(0, len(YT), batchSize):
        x = XT[i:i+batchSize]
        y = YT[i:i+batchSize]
        output = model(x).detach().numpy()
        pred = np.argmax(output, axis = 1)
        #print(pred)
        #print(y.numpy())
        numScore += np.sum((pred == y.numpy()))
        #print(numScore)
    if numOnly:
        print(numScore / len(YT))
    else:
        print("test acuuracy: {}".format(numScore / len(YT)))


        
def visualize(X, Y, model):
    W = model.weight.squeeze().detach().cpu().numpy()
    b = model.bias.squeeze().detach().cpu().numpy()

    delta = 0.001
    x = np.arange(X[:, 0].min(), X[:, 0].max(), delta)
    y = np.arange(X[:, 1].min(), X[:, 1].max(), delta)
    x, y = np.meshgrid(x, y)
    xy = list(map(np.ravel, [x, y]))

    z = (W.dot(xy) + b).reshape(x.shape)
    z[np.where(z > 1.0)] = 4
    z[np.where((z > 0.0) & (z <= 1.0))] = 3
    z[np.where((z > -1.0) & (z <= 0.0))] = 2
    z[np.where(z <= -1.0)] = 1

    plt.figure(figsize=(10, 10))
    plt.xlim([X[:, 0].min() + delta, X[:, 0].max() - delta])
    plt.ylim([X[:, 1].min() + delta, X[:, 1].max() - delta])
    plt.contourf(x, y, z, alpha=0.8, cmap="Greys")
    plt.scatter(x=X[:, 0], y=X[:, 1], c="black", s=10)
    plt.tight_layout()
    plt.show()



In [18]:
lr = 0.0005
reg = 0.0001

batchSize = 200
epoch = 10
    
model = nn.Linear(6, 5)
print (list(model.parameters()))
#print(model.weight)
#print(model.bias)
#print(torch.nn.Parameter.data)

#print(xx.dtype)
#print(model(xx) - torch.matmul(xx,torch.t(model.weight)))

train(X_train, y_train, X_val, y_val, model, lr, reg, batchSize, epoch)
test(X_test, y_test, model, batchSize)



[Parameter containing:
tensor([[ 0.0768, -0.0180, -0.2126,  0.1533, -0.2688, -0.0891],
        [ 0.1314,  0.0281, -0.4004,  0.2454,  0.3606, -0.0079],
        [ 0.0809,  0.3509, -0.2005,  0.2211, -0.2796,  0.1941],
        [ 0.1229,  0.0414, -0.4009,  0.1050, -0.2948,  0.1329],
        [ 0.3420, -0.1609, -0.0394,  0.0560, -0.0914, -0.1055]],
       requires_grad=True), Parameter containing:
tensor([-0.0583, -0.1855,  0.3728,  0.0816, -0.2952], requires_grad=True)]
Epoch:    0	loss: 0.014656129960667836
validation acuuracy: 0.4022184711839884
Epoch:    1	loss: 0.012883947887054768
validation acuuracy: 0.40358492082630015
Epoch:    2	loss: 0.012885020126355243
validation acuuracy: 0.4039707419017764
Epoch:    3	loss: 0.01288456614522072
validation acuuracy: 0.4035045414355759
Epoch:    4	loss: 0.012884820440628164
validation acuuracy: 0.4052889639096536
Epoch:    5	loss: 0.012884556586248376
validation acuuracy: 0.4051925086407845
Epoch:    6	loss: 0.012884760376282287
validation acuurac

In [20]:
print("Shuffle Begins")
model_shuffle = nn.Linear(6,5);
train(X_train_shuffle, y_train, X_val, y_val, model_shuffle, lr, reg, batchSize, epoch)
test(X_test, y_test, model_shuffle, batchSize)

Shuffle Begins
Epoch:    0	loss: 0.021986735594015615
validation acuuracy: 0.27714813921710474
Epoch:    1	loss: 0.02000272350030902
validation acuuracy: 0.2126999437344265
Epoch:    2	loss: 0.020006947031759877
validation acuuracy: 0.32818905232698337
Epoch:    3	loss: 0.02000956361466303
validation acuuracy: 0.17492163009404388
Epoch:    4	loss: 0.020011252811594795
validation acuuracy: 0.25116952013503735
Epoch:    5	loss: 0.020010805095085484
validation acuuracy: 0.2871151836669078
Epoch:    6	loss: 0.020001183927888508
validation acuuracy: 0.22845430431637329
Epoch:    7	loss: 0.019999690859376588
validation acuuracy: 0.24928864239209067
Epoch:    8	loss: 0.02001058560196853
validation acuuracy: 0.2503335744715055
Epoch:    9	loss: 0.020013968041159177
validation acuuracy: 0.2502210433244916
test acuuracy: 0.25071939554698175


In [21]:
print(valSet.shape)
print(testSet.shape)
fullTestSet = np.concatenate((valSet,testSet),axis=0)
print(fullTestSet.shape)
np.random.shuffle(fullTestSet)
X_fullTest = fullTestSet[:,:MBON_NUM]
y_fullTest = fullTestSet[:,-1]-1 # to conform python index format
print(X_fullTest.shape)
print(y_fullTest.shape)

(62205, 7)
(62205, 7)
(124410, 7)
(124410, 6)
(124410,)


In [24]:
sampleSize = 1000

#test for 120 cases
for i in range (0,120,1):
    test(X_fullTest[i*sampleSize:(i+1)*sampleSize,:], y_fullTest[i*sampleSize:(i+1)*sampleSize], model, batchSize, True)

0.408
0.411
0.426
0.386
0.42
0.381
0.386
0.411
0.413
0.395
0.393
0.414
0.396
0.395
0.387
0.392
0.407
0.386
0.411
0.363
0.415
0.385
0.391
0.377
0.406
0.401
0.399
0.387
0.404
0.429
0.394
0.407
0.399
0.411
0.398
0.391
0.404
0.403
0.397
0.396
0.366
0.382
0.391
0.408
0.421
0.39
0.39
0.404
0.399
0.411
0.409
0.4
0.414
0.415
0.429
0.4
0.401
0.388
0.37
0.394
0.418
0.41
0.404
0.389
0.414
0.391
0.38
0.379
0.406
0.396
0.406
0.387
0.42
0.424
0.399
0.381
0.415
0.441
0.429
0.381
0.397
0.418
0.396
0.412
0.422
0.407
0.402
0.406
0.403
0.419
0.387
0.387
0.37
0.387
0.392
0.41
0.378
0.421
0.403
0.387
0.394
0.394
0.419
0.375
0.403
0.389
0.397
0.404
0.399
0.395
0.429
0.402
0.372
0.425
0.369
0.401
0.396
0.411
0.413
0.401


In [25]:
sampleSize = 1000

#test for 120 cases
for i in range (0,120,1):
    test(X_fullTest[i*sampleSize:(i+1)*sampleSize,:], y_fullTest[i*sampleSize:(i+1)*sampleSize], model_shuffle, batchSize, True)

0.258
0.257
0.25
0.274
0.24
0.247
0.258
0.261
0.264
0.253
0.261
0.257
0.249
0.264
0.242
0.278
0.248
0.245
0.245
0.263
0.246
0.234
0.263
0.234
0.269
0.273
0.23
0.246
0.261
0.236
0.265
0.26
0.256
0.236
0.204
0.258
0.251
0.236
0.253
0.245
0.241
0.265
0.252
0.269
0.255
0.243
0.234
0.274
0.263
0.233
0.236
0.256
0.26
0.237
0.256
0.26
0.267
0.229
0.26
0.243
0.246
0.246
0.262
0.244
0.248
0.248
0.252
0.264
0.246
0.263
0.242
0.256
0.259
0.269
0.226
0.237
0.236
0.251
0.244
0.236
0.258
0.251
0.217
0.256
0.261
0.278
0.271
0.259
0.27
0.247
0.239
0.241
0.26
0.249
0.246
0.251
0.246
0.263
0.259
0.233
0.262
0.218
0.241
0.241
0.244
0.251
0.246
0.223
0.281
0.231
0.28
0.243
0.246
0.249
0.245
0.245
0.24
0.255
0.243
0.249
